## 1. EDA simplificado:


### Librerias:
Primero importaremos las librerias que necesitaremos para hacer las comprobaciones del EDA.

In [ ]:
import pandas as pd

### Numero de instancias y variables:
Con el uso de pandas podemos ver cuantas instancias y variables hay gracias a shape. En este caso hay 11000 instancias y 17 variables.

In [ ]:
# Leemos el pickle.
df = pd.read_pickle('bank_02.pkl')

# 1. Determinar cuántas variables e instancias hay.
print(f"Instancias: {df.shape[0]}")
print(f"Variables: {df.shape[1]}")

Instancias: 11000
Variables: 17
